<a href="https://colab.research.google.com/github/RaianaSeixas/0_paper_one_revisited/blob/main/CEC14/lshade/LSHADE/LSHADEv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import scipy.stats
import random

In [2]:
cols=30
dim = cols
NPAR1 = 100
ITERATIONS = 1000
MAX=np.repeat(100,dim)
MIN=np.repeat(-100,dim)
popsize = NPAR1
H = 6
kH = 0 # kH is used to work with H  
terminal = 0.1
p=0.1
SF=[];SCR=[];MF=[];MCR=[];Xarq=[]

In [3]:
# HappyCat function
# global mimima 0 at (-1,-1,...,-1)
# Not convex and non separable it is usually evaluated at xi =[(-600,600),..(-600,600)]

def fobj(x): 
    alpha=1. / 8
    s = sum(x**2) 
    return ((s - len(x))**2)**alpha + (s / 2 + sum(x)) / len(x) + 0.5 

In [4]:
def LShade(MAX,MIN, popsize,fobj,setTUNE,best,fbest,fitness,X,Xarq,FES):

  SF,SCR,MF,MCR,p, terminal, Narquive,H,kH = setTUNE

  fmutant = []
  Xmutant = []
  fx=[]; fu=[] # to select the wij Lehmer Mean.
  # setTUNE = [SF,SCR,MF,MCR,p, terminal, Narquive,H] 
  dim = X.shape[1]
  #best_number = int(p*popsize)
  if (p*popsize<1):
    best_number=1
  else:
    best_number = int(p*popsize)
  justGoOUT=1

  if(justGoOUT==1):
    SCR =[];SF=[];
    for i in range(popsize):
      ri = random.randint(0,H-1) # line 7
      miF = MF[ri]
      miCR = MCR[ri]
      if(MCR[ri] == terminal):
        miCR = 0
        crossp = 0
      else:
        crossp = np.random.normal(miCR , 0.1)
      while (True):
        mut = scipy.stats.cauchy.rvs(loc=miF, scale=0.1)
        if (mut > 0):
          break
      if(mut > 1.0): mut =1.0
      if(crossp > 1.0): crossp =1
      if(crossp < 0): crossp = 0 

      ind = fitness.argsort()[range(best_number)] # find index of best p*popsize
      best_idx = random.choice(ind) # index of the best in p*popsize (random best)
      pbest = X[best_idx] # random best 
      idxs = [idx for idx in range(popsize) if idx != i]
      iused = np.random.choice(idxs, 1, replace = False)
      idxs = [idx for idx in range(popsize) if (idx != iused and idx != best_idx)]
      a  = X[iused]
      if(len(Xarq)>0):
        Xii = np.asarray(Xarq)
        Xnew = np.append(X[idxs,:],Xii,axis=0)
      else:
        Xnew=np.copy(X[idxs,:])

      idxs2 = [idx for idx in range(len(Xnew[:,0])) if 2 > 1]
      b  = Xnew[np.random.choice(idxs2, 1, replace = False)]

      mutant = X[i,:]+mut*(pbest-X[i,:]) + mut * (b - a)
      mutant = mutant.ravel()

      #trial = np.copy(mutant)
      trial = np.copy(X[i,:])

      jrand = random.randint(0,dim-1)

      for j in range(dim):
        rdn = np.random.rand()
        if(rdn < crossp or j == jrand) : 
          trial[j] = mutant[j] 
        else:
          trial[j] = X[i,j]

      mutant  = np.copy(trial) # just to use word mutant instead of trial
      
      for k in range(dim):
        if(mutant[k]>MAX[k]):
          mutant[k]=MAX[k]
        if(mutant[k]<MIN[k]):
          mutant[k]=MIN[k]

      Xmutant.append(mutant)

      FES = FES + 1
      fmut =fobj(mutant)
      fmutant.append(fmut)
    
    

      #for i in range(popsize):
      if( fmutant[i]  <= fitness[i]):
        Xold = X[i,:] 
        X[i,:] = Xmutant[i]
        fx.append(fitness[i])
        fu.append(fmutant[i])
        fitness[i]=fmutant[i]
        SCR.append(crossp)
        SF.append(mut)

        sizeARQ = np.array(Xarq).shape[0]
        if(sizeARQ < Narquive):
          Xarq.append(Xold)
        else:
          ri = random.randint(0, Narquive-1) # deleting individuals if necessary line 19
          Xarq[ri] = Xold

        if(fbest >= fmutant[i]):
          best = Xmutant[i]
          fbest = fmutant[i]
      else:
        X[i,:] = X[i,:] # it is not necessary this line but just to remeber it

    
    # memory update algorithm 1:
    if(kH > H-1): kH = 0

    if (len(SCR)>0):
      deltaF = np.abs(np.asarray(fu)-np.asarray(fx))
      if( sum(deltaF) > 0):
        w = deltaF/sum(deltaF)
      else:
        w =np.ones(len(fu))
      if(MCR[kH] == terminal or max(SCR)==0):
        MCR[kH] = terminal
      else:
        y = np.asarray(SCR)
        MCR[kH] = np.dot(w,y**2)/np.dot(w,y)
      y = np.asarray(SF)
      MF[kH] = np.dot(w,y**2)/np.dot(w,y)
      kH = kH + 1

    else:
      MCR = MCR
      MF = MF

  setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H,kH]


  y=fitness


  XY= np.c_[X,y] #concatena x e y em 2 colunas            
  XYsorted = XY[XY[:,-1].argsort()] #Ordena a partir da last col(Y) for all row
  X=XYsorted[:,0:dim]
  XY=XYsorted
  fitness = XYsorted[:,-1]  
  BEST=X[0,:]
  FOBEST=fitness[0]
  BEST_XY =np.append(BEST,FOBEST)
  
  return fitness,X,BEST,FOBEST,XY,BEST_XY,FES,Xarq,setTUNE


In [5]:

X=np.zeros((NPAR1,cols)) # X1 is Frannk population
for i in range(NPAR1):
  for j in range(cols):
    r=np.random.random()
    X[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

FES = 0
X1 = np.copy(X)
fit1 = np.asarray([fobj(ind) for ind in X1])
FES = FES + NPAR1
best_idx = np.argmin(fit1)
best = X1[best_idx]
fbest = fit1[best_idx] 
BEST1,FOBEST1 = best, fbest

In [6]:
for i in range(H):
  MCR.append(0.5);MF.append(0.5) # line 3
Narquive = popsize
setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H,kH]

In [7]:
for i in range(ITERATIONS):
  
  fit1,X1,BEST1,FOBEST1,XY,BEST_XY,FES,Xarq,setTUNE= LShade(MAX,MIN, popsize,fobj,setTUNE,best,fbest,fit1,X1,Xarq,FES)
  print('i=',i,FOBEST1)


i= 0 802.6335876684275
i= 1 791.5470337694429
i= 2 597.5061890795884
i= 3 597.5061890795884
i= 4 475.1618092297793
i= 5 475.1618092297793
i= 6 411.0015827745234
i= 7 303.4543283093099
i= 8 225.99122814282256
i= 9 225.99122814282256
i= 10 223.9186672810468
i= 11 223.9186672810468
i= 12 194.00827476218132
i= 13 194.00827476218132
i= 14 173.25793206563904
i= 15 147.22176683448836
i= 16 91.40480106225104
i= 17 87.15624573319363
i= 18 75.00506508907324
i= 19 69.15713233000274
i= 20 61.40539838368017
i= 21 60.77213959299087
i= 22 50.18815251343222
i= 23 50.18815251343222
i= 24 47.86700030081223
i= 25 36.67872055581834
i= 26 31.800312052430527
i= 27 29.681823741018267
i= 28 29.681823741018267
i= 29 27.445186352371202
i= 30 23.830795937191635
i= 31 19.015218269804425
i= 32 19.015218269804425
i= 33 17.231149730108196
i= 34 12.006812322015618
i= 35 12.006812322015618
i= 36 12.006812322015618
i= 37 9.088307198613947
i= 38 7.767906027157977
i= 39 7.767906027157977
i= 40 7.767906027157977
i= 41 7.4

In [8]:
X1.shape

(100, 30)

In [9]:
fobj(X1[0,:])

0.2661371715714197

In [10]:
fit1

array([0.26613717, 0.2807132 , 0.28133761, 0.29136016, 0.29758893,
       0.30386934, 0.31533933, 0.33550378, 0.34023943, 0.34789233,
       0.3496116 , 0.35071381, 0.35103257, 0.35271241, 0.35530688,
       0.35590852, 0.36476712, 0.36936293, 0.37380244, 0.3754957 ,
       0.37797859, 0.37998281, 0.38156343, 0.38371213, 0.38946999,
       0.39252262, 0.39533406, 0.39833485, 0.39942121, 0.40387469,
       0.40559654, 0.40589078, 0.40644049, 0.40671909, 0.40785757,
       0.41128825, 0.41222034, 0.41237487, 0.41420343, 0.41565891,
       0.41643258, 0.41779353, 0.42066465, 0.42221214, 0.42263554,
       0.42391283, 0.43301899, 0.4340655 , 0.43529129, 0.43532533,
       0.43718335, 0.43835565, 0.4487132 , 0.45011027, 0.45045138,
       0.45098028, 0.45381743, 0.46066888, 0.46150711, 0.46275888,
       0.46409972, 0.46845413, 0.47087736, 0.47179832, 0.47406394,
       0.47497215, 0.47587604, 0.47595069, 0.47770671, 0.47877493,
       0.48102403, 0.49332729, 0.49332953, 0.49509095, 0.49568

In [11]:
setTUNE

[[],
 [],
 [0.031386842088052515,
  0.0626266478013553,
  0.0005495340860815073,
  0.017409781720822956,
  0.7957947489427593,
  0.00616283296722548],
 [0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
 0.1,
 0.1,
 100,
 6,
 2]

In [12]:
setTUNE[2]# =[SF,SCR,MF,MCR,p, terminal, Narquive,H,kH]

[0.031386842088052515,
 0.0626266478013553,
 0.0005495340860815073,
 0.017409781720822956,
 0.7957947489427593,
 0.00616283296722548]

In [13]:
setTUNE[3]# =[SF,SCR,MF,MCR,p, terminal, Narquive,H,kH]

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]